In [ ]:
pip install selenium
pip install webdriver-manager

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import openai

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://finance.yahoo.com/topic/stock-market-news")

# Scroll to the end of the page to ensure all dynamic content is loaded
body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(10): # We can also increase the number to have more scroll - i.e. more data
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)  # Wait for the page to load

news_titles = driver.find_elements(By.CSS_SELECTOR, 'h3[class*="Mb(5px)"]') 
news_descriptions = driver.find_elements(By.CSS_SELECTOR, 'p[class*="Fz(14px)"]')
articles_data = []

for title, description in zip(news_titles, news_descriptions):
    articles_data.append({"title": title.text, "description": description.text})

with open('articles_data.json', 'w', encoding='utf-8') as f:
    json.dump(articles_data, f, ensure_ascii=False, indent=4)

driver.quit()


In [ ]:
persona = "You will response as if you are an optimistic, expert stock market analyst on Twitter. You have a deep understanding of financial markets. Respond in the manner of a catchy Tweet, with relevant hashtags, if needed. Limit your response to 280 characters maximum."
scraping = "Focus primarily on the following articles given in the following context to answer any questions: Context:" +  article_string
context = "Give me a Tweet"
response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages=[
           {"role": "system", "content": persona},
           {"role": "system", "content": scraping},
           {"role": "user", "content": context},
        ]
     )
print (response.choices[0].message.content)